In [2]:
import re
import pandas as pd
import tqdm, time
file_dir = '~/Documents/Materials/iron/'

#### 1. generate a txt file with format chrN:12345678-12345678 using chromosome and hg19_position from file all_variants_and_proxies.csv

In [5]:
df = pd.read_csv(file_dir+'all_variants_and_proxies_new.csv')
df['format'] = 'chr'+df.variant2_chromosome.astype(str)+':'+df.variant2_hg19_position.astype(str)+'-'+df.variant2_hg19_position.astype(str)
df['format'].to_csv(file_dir+'variants_hg19_new.txt', index=False) 

#### 2. convert hg19 coordinates to hg38 coordinates using liftover tool: 
https://genome.ucsc.edu/cgi-bin/hgLiftOver

#### 3. rename the file downloaded from the tool as variants_hg19.bed and then add it to all_variants_and_proxies.csv

In [7]:
df0 = pd.read_csv(file_dir+'variants_hg38_new.bed', header=None) 
df['hg38_position'] = df0[0].str.extract('\:(\d+)\-') 
df.to_csv(file_dir+'all_variants_and_proxies_new.csv', index=False)

#### 4. obtain phecode_map from PheWAS package in R
See phecode_map.R for detail
#### 5. label a PheCode if it is sex-specified
Download sex-specified PheCode dictionary from https://phewascatalog.org/files/phecode_definitions1.2.csv.zip

In [411]:
phecode_map = pd.read_csv(file_dir+'phecode_map.csv') 
phecode_def = pd.read_csv(file_dir+'phecode_definitions1.2.csv') 
phecode_list = phecode_map.phecode.unique().tolist()
phecode_dict = pd.DataFrame({'phecode':phecode_list})
phecode_dict = phecode_dict.merge(phecode_def[['phecode','sex']], 'left')

In [416]:
phecode_sex = phecode_dict[(~phecode_dict.sex.isna())&(phecode_dict.sex != 'Both')]
# in total 166 sex-specified phecodes
phecode_sex.to_csv(file_dir+'phecode_sex.csv', index=None)

#### 6. upload the file to green bucket for adding AF and finding out the SNPs we have in Finngen

gsutil cp ~/Documents/Materials/iron/all_variants_and_proxies_new.csv gs://given_link

#### 7. upload the new file with selected SNPs and rerun the process for the final version:

In [3]:
df = pd.read_csv(file_dir+'finngen_all_variants_and_proxies_new_final.csv')
df['format'] = 'chr'+df.variant2_chromosome.astype(str)+':'+df.variant2_hg19_position.astype(str)+'-'+df.variant2_hg19_position.astype(str)
df['format'].to_csv(file_dir+'variants_hg19_new_final.txt', index=False) 

##### lift over: https://genome.ucsc.edu/cgi-bin/hgLiftOver

In [4]:
df0 = pd.read_csv(file_dir+'variants_hg38_new_final.bed', header=None) 
df['hg38_position'] = df0[0].str.extract('\:(\d+)\-') 
df.to_csv(file_dir+'all_variants_and_proxies_new_final.csv', index=False)

gsutil cp ~/Documents/Materials/iron/finngen_all_variants_and_proxies_new_final.csv gs://given_link

- gsutil cp ~/Documents/Materials/iron/phecode_map.csv gs://given_link

- gsutil cp ~/Documents/Materials/iron/phecode_sex.csv gs://given_link

#### 8. download summary statistics
download the final results from green box to local machine for 2SMR analysis

gsutil cp gs://given_link/results.csv ~/Documents/Materials/iron/

chr6_26090957_A_T, chr6_26092913_G_A, chr6_26104404_T_G

- '401': 'I9_HYPTENS',
- '280': 'D3_ANAEMIA_IRONDEF',
- '285': 'D3_ANAEMIANAS',
- '275': 'E4_MINERAL_MET'

In [45]:
phecode_def = pd.read_csv(file_dir+'phecode_definitions1.2.csv') 

In [49]:
phecode_def[phecode_def.phecode == 275.]

,phecode,phenotype,phecode_exclude_range,sex,rollup,leaf,category_number,category
356,275.0,Disorders of mineral metabolism,275-275.99,NaN,1,0,3,endocrine/metabolic


In [31]:
results = pd.read_csv(file_dir+'results1.csv', dtype={5:'str'})
results = results.rename(columns={'Unnamed: 0':'snp'})

In [33]:
results['Coef.'] = results['Coef.']*(-1)

In [34]:
results.to_csv(file_dir+'results.csv', index=None)

In [30]:
results[results.snp.str.startswith('chr6_2609')].snp.unique()

array(['chr6_26090951_C_G', 'chr6_26092913_G_A', 'chr6_26098246_T_C',
       'chr6_26090957_A_T'], dtype=object)

In [3]:
af = pd.read_csv(file_dir+'select_snp_af.csv')

In [62]:
af[af.sandbox_format == 'chr6_26104404_T_G']

,chromosome,hg19_position,rsid,ea,nea,eaf,hg38_position,finn_format,finn_af,finn_rsid,finn_ref,finn_alt,match,finn_hg38,sandbox_format,sandbox_af
72,6,26104632,rs198851,T,G,0.15148,26104404,6:26104404:T:G (rs198851),0.89,rs198851,T,G,True,26104404.0,chr6_26104404_T_G,0.89002
73,6,26104632,rs198851,G,T,0.84852,26104404,6:26104404:T:G (rs198851),0.89,rs198851,T,G,True,26104404.0,chr6_26104404_T_G,0.89002


In [4]:
af.to_excel(file_dir+'select_snp_af.xlsx', index = False)

In [35]:
results.snp.unique()

array(['chr6_26090957_A_T', 'chr6_26092913_G_A', 'chr6_26104404_T_G'],
      dtype=object)

In [37]:
results[results.outcome == '275']

,snp,Coef.,Std.Err.,z,P>|z|,outcome,n_cases,n_cohort
268,chr6_26090957_A_T,-0.084252,0.102752,0.819953,4.122429e-01,275,2495,377360
1684,chr6_26092913_G_A,0.925677,0.050119,-18.469530,3.632327e-76,275,2495,377360
3100,chr6_26104404_T_G,-0.145452,0.042948,3.386708,7.073652e-04,275,2495,377360


In [38]:
results.iloc[0]

snp         chr6_26090957_A_T
Coef.               -0.024645
Std.Err.             0.025668
z                    0.960125
P>|z|                0.336992
outcome                   008
n_cases                 42553
n_cohort               377360
Name: 0, dtype: object